In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np
import pandas as pd

import pydicom as dicom
import os
import cv2
import PIL # optional
import shutil
import tensorflow as tf
import shutil
import SimpleITK
from tqdm import tqdm

In [ ]:
# Loading input data file in png format to dataframes
#train images in png format and stored in annotation format in dataframe
import os
kf = pd.DataFrame(columns=['data_type','StudyInstanceUID','image_file_ID','image_file_path'])
for dirname, _, filenames in os.walk('/kaggle/input/siim-covid19-detection-dicom-to-png-conversion'):
    for filename in filenames:
        
        std_id = filename.split('_')[0]
        #f_name_id = filename.split('_')[1].split('.')[0]
        dat_type = dirname.split('/')[-1]
        img_path = os.path.join(dirname, filename)
        
        kf.loc[kf.shape[0]] = [dat_type,std_id,filename,img_path]


png_data = pd.DataFrame(kf.drop([0,1,2,3,4],axis=0).values,columns=kf.columns)
png_data['image_file_ID'] = png_data.image_file_ID.str[13:-4]
png_data

# creating train and test datframes where all the image_study and studyInstanceUID are stored
png_test = png_data[png_data['data_type'] == 'test']
png_train = png_data[png_data['data_type'] == 'train']

In [ ]:
sample = pd.read_csv("/kaggle/input/siim-covid19-detection/sample_submission.csv")
train_image = pd.read_csv("/kaggle/input/siim-covid19-detection/train_image_level.csv")
train_study = pd.read_csv("/kaggle/input/siim-covid19-detection/train_study_level.csv")

# class label colum and case study id_number columns are created

train_study['classification_class'] = np.zeros((train_study.shape[0],1))
for i in range(0,train_study.shape[0]):
    train_study['classification_class'][i] = train_study.iloc[i].values[1:-1]

# StudyInstanceUID 
train_study['StudyInstanceUID'] = train_study.id.str[0:-6]
train_study

In [ ]:
itr = 0
cl = []
for i in train_study['classification_class'].values:
    if i[0]==1:
        cl.append('Negative')
    elif i[1]==1:
        cl.append('Typical')
    elif i[2]==1:
        cl.append('Indeterminate')
    elif i[3]==1:
        cl.append('Atypical')
    else:
        a=1
    itr = itr+1

train_study['class_type'] = cl
train_study

In [ ]:
# This data frame is for the object localisation and classification combined activities

train_Study_image = pd.merge(train_image,train_study,on='StudyInstanceUID',how="inner")
#
train_Study_image.columns = ['Image_id', 'boxes', 'label', 'StudyInstanceUID', 'Study_id',
       'Negative for Pneumonia', 'Typical Appearance',
       'Indeterminate Appearance', 'Atypical Appearance',
       'classification_class','class_type']

train_Study_image

In [ ]:
std_img_df = train_Study_image.dropna(subset=['boxes'],axis=0)
std_img_df

In [ ]:
dcm_df =pd.read_csv('/kaggle/input/dicom-df/Siim_Covid_dcm_df.csv')
dcm_df.set_index('image_file_id',inplace=True)
dcm_df

In [ ]:
def scale_factor(dcm_image_path):
               
    ds_array = SimpleITK.ReadImage(dcm_image_path)  
    img_array = SimpleITK.GetArrayFromImage(ds_array)  
    x_fct = 256 /img_array.shape[2] # divide by width
    y_fct = 256 /img_array.shape[1] # divide by height
    
    return [x_fct,y_fct]

In [ ]:

cols = ['Study_image_id','images_label','Case_label','width','height','Xmin','Ymin','Xmax','Ymax','image_file_path']
trn_csv = pd.DataFrame(columns=cols)
itr = 0

for r in np.arange(std_img_df.shape[0]):
    row = std_img_df.iloc[r]
    n = int(len(row.label.split(' '))/6)
    
    dcm_img_path = dcm_df.loc[row.Image_id[0:-6]].image_file_path
    scl_fctr = scale_factor(dcm_img_path)
    
    for i in np.arange(n):
        k = i*6
        box = row.label.split(' ')[k:k+6]
        opac = box[0]
        xmin = float(box[2])*scl_fctr[0]
        ymin = float(box[3])*scl_fctr[1]
        w_h = 256
        xmax = xmin + (float(box[4]))*scl_fctr[0]
        ymax = ymin + (float(box[5]))*scl_fctr[1]
        
        c_mid = row.StudyInstanceUID + '_' + row.Image_id[0:-6]
        im_path = '/kaggle/input/siim-covid19-detection-dicom-to-png-conversion/siim-covid19-detection/train/' + c_mid + '.png'
        case_label = row.class_type
        trn_csv.loc[trn_csv.shape[0]] = [c_mid,opac,case_label,w_h,w_h,xmin,ymin,xmax,ymax,im_path]

In [ ]:
trn_csv.head()

In [ ]:
tr2 = trn_csv.drop('images_label',axis = 1)
tr1 = trn_csv.drop('Case_label',axis = 1)

In [ ]:
tr1.columns = ['Study_image_id', 'label', 'width', 'height', 'Xmin', 'Ymin',
       'Xmax', 'Ymax', 'image_file_path']

tr2.columns = ['Study_image_id', 'label', 'width', 'height', 'Xmin', 'Ymin',
       'Xmax', 'Ymax', 'image_file_path']

In [ ]:
train_final_csv = pd.concat([tr1,tr2],axis=0,ignore_index=True)
train_final_csv

In [ ]:
train_final_csv.to_csv('final_train_csv.csv')

In [ ]:
dft = pd.read_csv('/kaggle/input/siim-train-csv/final_train_csv.csv')
dft.drop(['Unnamed: 0'],axis = 1,inplace=True)

In [ ]:
dft